# EMA Project Notebook
__Name:__ Daniel Smith

__PI:__ A7603242

In [1]:
# import the required libraries
import pandas as pd
import scipy.stats
import numpy as np
import pymongo
import bson
import collections
from matplotlib import pyplot
import seaborn as sns

In [2]:
# make a folder for storing my working files as I go along.
!mkdir -p data/dcs283

In [3]:
!ls data/dcs283

ks2_cleaned.csv


# Contents

Use these links to jump to a section.

[Setting up mongo](#mongo)
[Data preparation](#preparation)
 - [Importing the KS2 data](#importing_ks2)
 - [Importing the KS4 data](#importing_ks4)
 - [OpenRefine](#openrefine)

[Q1, KS4 Investigation](#q1)

[Application of Machine Learning](#machine_learning)

[Q2, KS2 - KS4 Investigation](#q2)

[Cleanup remove the database](#cleanup)

# Setting up mongo
<a name="mongo"></a>

In [4]:
# set up a connection to mongodb server
client = pymongo.MongoClient('mongodb://localhost:27351')

In [5]:
# uncomment to remove the database if needed
client.drop_database('schools_db')
client.database_names()

['accidents', 'admin', 'local']

In [6]:
# setup a schools_db database on mongo
db = client.schools_db

# Data preparation
<a name="preparation"></a>

Before we can investigate the data we will need to have a quick look at it, determine what cleaning, if any, is needed.  Carry out the cleaning and store it for access in tn appropriate form.  

However before doing anything I will import the KS2 data in the same way as was done in `dcs283_TMA02_Question2b-pd`  I will then store the resultant dataframe into mongo for analysis later on.

## Importing the KS2 data
<a name="importing_ks2"></a>

All of this section is the same as in the `TMA02_Question2b-pd` notebook.

***
__ ----------- Beginning of TMA02 code -----------  __

### Import the LEA data

In [7]:
leas_df = pd.read_csv('data/2015-2016/la_and_region_codes_meta.csv')
leas_df.head()

,LEA,LA Name,REGION,REGION NAME
0,841,Darlington,1,North East A
1,840,County Durham,1,North East A
2,805,Hartlepool,1,North East A
3,806,Middlesbrough,1,North East A
4,807,Redcar and Cleveland,1,North East A


### Import the KS2 data
Most of the field names are given in the `ks2_meta` file, so we'll use that to keep track of the types of various columns.

In [8]:
ks2cols = pd.read_csv('data/2015-2016/ks2_meta.csv')
ks2cols['Field Name'] = ks2cols['Field Name'].apply(lambda r: r.strip(),)
ks2cols

,Column,Field Name,Label/Description
0,1,RECTYPE,Record type (1=mainstream school; 2=special sc...
1,2,ALPHAIND,Alphabetic index
2,3,LEA,Local authority number
3,4,ESTAB,Establishment number
4,5,URN,School unique reference number
5,6,SCHNAME,School/Local authority name
6,7,ADDRESS1,School address (1)
7,8,ADDRESS2,School address (2)
8,9,ADDRESS3,School address (3)
9,10,TOWN,School town


Some columns contain integers, but _**pandas**_ will treat any numeric column with `na` values as `float64`, due to NumPy's number type hierarchy. 

In [9]:
int_cols = [c for c in ks2cols['Field Name'] 
            if c.startswith('T')
            if c not in ['TOWN', 'TELNUM', 'TKS1AVERAGE']]
int_cols += ['RECTYPE', 'ALPHAIND', 'LEA', 'ESTAB', 'URN', 'URN_AC', 'ICLOSE']
int_cols += ['READ_AVERAGE', 'GPS_AVERAGE', 'MAT_AVERAGE']

Some columns contain percentages. We'll convert these to floating point numbers on import.

Note that we also need to handle the case of `SUPP` and `NEW` in the data.

In [10]:
def p2f(x):
    if x.strip('%').isnumeric():
        return float(x.strip('%'))/100
    elif x in ['SUPP', 'NEW', 'LOWCOV', 'NA', '']:
        return 0.0
    else:
        return x

These are the columns to try to convert from percentages. Note that we can be generous here, as columns like PCODE (postcode) will return the original value if the conversion fails.

In [11]:
percent_cols = [f for f in ks2cols['Field Name'] if f.startswith('P')]
percent_cols += ['WRITCOV', 'MATCOV', 'READCOV'] 
percent_cols += ['PTMAT_HIGH', 'PTREAD_HIGH', 'PSENELSAPK', 'PSENELK', 'PTGPS_HIGH']
percent_converters = {c: p2f for c in percent_cols}

In [12]:
ks2_df = pd.read_csv('data/2015-2016/england_ks2final.csv', 
                   na_values=['SUPP', 'NEW', 'LOWCOV', 'NA', ''],
                   converters=percent_converters)

Drop the summary rows, keeping just the rows for mainstream and special schools.

In [13]:
ks2_df = ks2_df[(ks2_df['RECTYPE'] == 1) | (ks2_df['RECTYPE'] == 2)]

Convert everything to numbers, if possible.

In [14]:
ks2_df = ks2_df.apply(pd.to_numeric, errors='ignore')

Merge the LEA data into the school data

In [15]:
ks2_df = pd.merge(ks2_df, leas_df, on=['LEA'])
ks2_df.head().T

,0,1,2,3,4
RECTYPE,1,1,1,1,1
ALPHAIND,53372,11156,11160,11256,16366
LEA,201,202,202,202,202
ESTAB,3614,3323,3327,2842,2184
URN,100000,100028,100029,130342,100013
SCHNAME,Sir John Cass's Foundation Primary School,"Christ Church Primary School, Hampstead",Christ Church School,Christopher Hatton Primary School,Edith Neville Primary School
ADDRESS1,St James's Passage,Christ Church Hill,Redhill Street,38 Laystall Street,174 Ossulston Street
ADDRESS2,Duke's Place,NaN,Camden,NaN,NaN
ADDRESS3,NaN,NaN,NaN,NaN,NaN
TOWN,London,London,London,London,London


__ ----------- END of TMA02 code -----------  __
***

### Convert and store the KS2 dataframe into Mongo for use later

In [16]:
# set up a collection for the ks2 results dataframe
ks2 = db.ks2_results

In [17]:
# convert the dataframe into a list of dicts and store in Mongo

# the 'results' argument is needed to get a list of dicts
ks2.insert_many(ks2_df.to_dict('records'))

# snippet reference is from:
# https://stackoverflow.com/questions/33979983/insert-rows-from-pandas-dataframe-into-mongodb-collection-as-individual-document

In [18]:
# check we got them all
ks2.find().count(), len(ks2_df)

(16162, 16162)

In [19]:
ks2.find_one()

{'ADDRESS1': "St James's Passage",
 'ADDRESS2': "Duke's Place",
 'ADDRESS3': nan,
 'AGERANGE': '3-11',
 'ALPHAIND': 53372.0,
 'BELIG': 16.0,
 'CONFEXAM': nan,
 'DIFFN_MATPROG': 2.7,
 'DIFFN_READPROG': 0.6,
 'DIFFN_RWM_EXP': 23.0,
 'DIFFN_RWM_HIGH': -7.0,
 'DIFFN_WRITPROG': 0.2,
 'ESTAB': 3614.0,
 'GELIG': 12.0,
 'GPS_AVERAGE': 106.0,
 'GPS_AVERAGE_FSM6CLA1A': 105.0,
 'GPS_AVERAGE_H': 110.0,
 'GPS_AVERAGE_L': nan,
 'GPS_AVERAGE_M': 105.0,
 'GPS_AVERAGE_NotFSM6CLA1A': 107.0,
 'ICLOSE': 0.0,
 'LA Name': 'City of London',
 'LEA': 201.0,
 'MATCOV': 1.0,
 'MATPROG': 3.0,
 'MATPROG_B': 2.9,
 'MATPROG_B_LOWER': 0.3,
 'MATPROG_B_UPPER': 5.5,
 'MATPROG_EAL': 3.1,
 'MATPROG_EAL_LOWER': 0.6,
 'MATPROG_EAL_UPPER': 5.6,
 'MATPROG_FSM6CLA1A': 2.9,
 'MATPROG_FSM6CLA1A_LOWER': -0.1,
 'MATPROG_FSM6CLA1A_UPPER': 5.9,
 'MATPROG_G': 3.1,
 'MATPROG_G_LOWER': 0.1,
 'MATPROG_G_UPPER': 6.1,
 'MATPROG_H': 0.4,
 'MATPROG_H_LOWER': -3.9,
 'MATPROG_H_UPPER': 4.7,
 'MATPROG_L': nan,
 'MATPROG_LOWER': 1.0,
 'MATPROG

In [20]:
## Initial look at the KS4 results dataset

In [21]:
ks2.find_one()['GPS_AVERAGE_L']

nan

In [22]:
ks2.find({'GPS_AVERAGE_L': np.nan}).count()

11940

Looks good.  We will need to watch out for the NaN values though.

In [23]:
ks2.find({'GPS_AVERAGE_L': np.nan}).count()

11940

In [24]:
ks2.find_one()

{'ADDRESS1': "St James's Passage",
 'ADDRESS2': "Duke's Place",
 'ADDRESS3': nan,
 'AGERANGE': '3-11',
 'ALPHAIND': 53372.0,
 'BELIG': 16.0,
 'CONFEXAM': nan,
 'DIFFN_MATPROG': 2.7,
 'DIFFN_READPROG': 0.6,
 'DIFFN_RWM_EXP': 23.0,
 'DIFFN_RWM_HIGH': -7.0,
 'DIFFN_WRITPROG': 0.2,
 'ESTAB': 3614.0,
 'GELIG': 12.0,
 'GPS_AVERAGE': 106.0,
 'GPS_AVERAGE_FSM6CLA1A': 105.0,
 'GPS_AVERAGE_H': 110.0,
 'GPS_AVERAGE_L': nan,
 'GPS_AVERAGE_M': 105.0,
 'GPS_AVERAGE_NotFSM6CLA1A': 107.0,
 'ICLOSE': 0.0,
 'LA Name': 'City of London',
 'LEA': 201.0,
 'MATCOV': 1.0,
 'MATPROG': 3.0,
 'MATPROG_B': 2.9,
 'MATPROG_B_LOWER': 0.3,
 'MATPROG_B_UPPER': 5.5,
 'MATPROG_EAL': 3.1,
 'MATPROG_EAL_LOWER': 0.6,
 'MATPROG_EAL_UPPER': 5.6,
 'MATPROG_FSM6CLA1A': 2.9,
 'MATPROG_FSM6CLA1A_LOWER': -0.1,
 'MATPROG_FSM6CLA1A_UPPER': 5.9,
 'MATPROG_G': 3.1,
 'MATPROG_G_LOWER': 0.1,
 'MATPROG_G_UPPER': 6.1,
 'MATPROG_H': 0.4,
 'MATPROG_H_LOWER': -3.9,
 'MATPROG_H_UPPER': 4.7,
 'MATPROG_L': nan,
 'MATPROG_LOWER': 1.0,
 'MATPROG

In [25]:
# to save resources we can now remove delete the ks2_df
del ks2_df

## Importing the KS4 results dataset
<a name="importing_ks4"></a>

Before we can investigate the data we will need to have a look at it, determine what cleaning if any needs to be done, and store it for access in an appropriate form.

### Initial look at the KS4 results dataset
Let's have a quick look at the data we will be looking at for the EMA.

In [26]:
!head -5 'data/2015-2016/england_ks4final.csv'

In [27]:
!wc -l 'data/2015-2016/england_ks4final.csv'

5489 data/2015-2016/england_ks4final.csv


The dataset has 5489 rows of data, there appears to be a large number of columns and a lot of codes that I'll need to look up.  There are a also a number of `NA` and `NP` values that could be missing data.  As well as this results data I will need to find and import the relevent metadata file.

Looking through the data/2015-2016 folder there are a number of files that have information on these codes.

In [28]:
!ls data/2015-2016/

abbreviations.xlsx	    england_swf.csv
abs_meta.csv		    england_vaqual.csv
census_meta.csv		    england_vasubj.csv
england_abs.csv		    keto_setup.xlsx
england_census.csv	    ks2_meta.csv
england_cfrfull.xlsx	    ks4_meta.csv
england_ks2final.csv	    ks4_meta_methodology.csv
england_ks4final.csv	    ks4-pupdest_meta.csv
england_ks4-pupdest.csv     ks5_meta.csv
england_ks4underlying.xlsx  ks5-studest_meta.csv
england_ks5final.csv	    la_and_region_codes_meta.csv
england_ks5-studest.csv     sixth_form_centres_and_consortia_meta.xlsx
england_ks5underlying.xlsx  spine_meta.csv
england_spine.csv	    swf_meta.csv


There is an abbreviations file, stored as an xlsx file.  I'll have a quick glance at it in excel.  Having looked the abbreviation up in the abbreviations file we can see that they have the following meanings:

- _NA_: Not applicable
- _NP_: Not Published
- _NE_: No entries
- _SUPP_: Suppressed (5 or fewer in cohort)
- _LOWCOV_: Low coverage (less than 50% of the cohort
- _NEW_: New institution

The abbreviations file also has listings of all the school types (NFTYPE) that I will need.  I'll grab that for use later on.

## Importing the abbreviations file

In [29]:
# read in the abbreviations file
abbr_df = pd.read_excel('data/2015-2016/abbreviations.xlsx')
abbr_df

,2016 KS4 and KS5/16-18 Performance Tables,Unnamed: 1,Unnamed: 2
0,Abbreviations used in the csv and excel Downlo...,NaN,NaN
1,NaN,NaN,NaN
2,Institution type (NFTYPE):,NaN,NaN
3,AC,Sponsored academy,NaN
4,ACC,Academy converter - mainstream,NaN
5,AC1619,Academy 16-19 sponsor led,NaN
6,ACC1619,Academy 16-19 converter,NaN
7,ACCS,Academy converter - special school,NaN
8,ACS,Sponsored special academy,NaN
9,CTC,City technology college,NaN


We can see that the school types are rows 2-25, I'll store them as a dict for reference later on.

In [30]:
# relabel the columns
abbr_df.columns = ['label', 'expanded', 'not_needed']

In [31]:
nftypes = {}
for index, row in abbr_df[3:26].iterrows():
    nftypes[row['label'].strip()] = row['expanded'].strip()
    
nftypes

{'AC': 'Sponsored academy',
 'AC1619': 'Academy 16-19 sponsor led',
 'ACC': 'Academy converter - mainstream',
 'ACC1619': 'Academy 16-19 converter',
 'ACCS': 'Academy converter - special school',
 'ACS': 'Sponsored special academy',
 'CTC': 'City technology college',
 'CY': 'Community school',
 'CYS': 'Community special school',
 'F': 'Free school - mainstream',
 'F1619': 'Free school - 16-19',
 'FD': 'Foundation school',
 'FDS': 'Foundation special school',
 'FESI': 'Further Education Sector Institution',
 'FS': 'Free school - special',
 'FSS': 'Studio school',
 'FUTC': 'UTC (university technical college)',
 'IND': 'Independent school',
 'INDSPEC': 'Independent special school',
 'MODFC': 'College funded by Ministry of Defence',
 'NMSS': 'Non-maintained special school',
 'VA': 'Voluntary aided school',
 'VC': 'Voluntary controlled school'}

And, while we have the abbreviations available I'll store the missing value types for reference later on if needed.

In [32]:
missing_types = {}
for i, r in abbr_df[45:51].iterrows():
    missing_types[r['label']] = r['expanded']

missing_types

{'NEW': 'New institution',
 'NE': 'No entries',
 'SUPP': "Indicates that a school or college's figures have been suppressed because there are 5 or fewer pupils in the cohort",
 'LOWCOV': 'Low coverage: indicates that a school’s Progress 8 or value added measures have been suppressed because coverage is less than 50% of the cohort',
 'NP': 'Not published - for example we do not publish Progress 8 data for independent schools and independent special schools, or breakdowns by disadvantaged and non-disadvantaged pupils for independent schools, independent special schools and non-maintained special schools.',
 nan: 'Not applicable: figures are either not available for the year in question, or the data field is not applicable to this school or college'}

I can now delete the abbr_df as it won't be needed.

In [33]:
del abbr_df

## Importing the KS4 Metadata file

In order to analyse the data we need to be able to reference the columns and the codes they represent.  I'll import the KS4_meta.csv file into the database and use it to help me understand the data in the KS4 results dataset.

In [34]:
!head -5 data/2015-2016/ks4_meta.csv

372,TAVENT_GNFSM6CLA1A_15_PTQ_EE,Average number of GCSE entries per non-disadvantaged pupil in 2015,,,,Yes,,,

In [35]:
!wc -l data/2015-2016/ks4_meta.csv

0 data/2015-2016/ks4_meta.csv


0 lines.. I'll try loding directly into Mongo

In [36]:
!/usr/bin/mongoimport --port 27351 --drop --db schools_db --collection ks4_meta \
    --type csv --headerline --ignoreBlanks \
    --file data/2015-2016/ks4_meta.csv

2018-05-28T22:12:58.545+0000	Failed: fields cannot be identical: '' and ''
2018-05-28T22:12:58.545+0000	imported 0 documents


Clearly there is an issue with the import.  I'll try importing it into a dataframe.

In [37]:
ks4_meta_df = pd.read_csv('data/2015-2016/ks4_meta.csv')
ks4_meta_df.head()

,Column,Metafile heading,Metafile description,Methodology changes,Null field for special schools,Null field for local authority records,Null field for National (all schools) records,Null field for National (maintained schools) records,Unnamed: 8,Unnamed: 9
0,1,RECTYPE,Record type (1=mainstream school; 2=special sc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ALPHAIND,Alphabetic sorting index,NaN,NaN,Yes,Yes,Yes,NaN,NaN
2,3,LEA,Local authority code (see separate list of loc...,NaN,NaN,NaN,Yes,Yes,NaN,NaN
3,4,ESTAB,Establishment number,NaN,NaN,Yes,Yes,Yes,NaN,NaN
4,5,URN,School Unique Reference Number,NaN,NaN,Yes,Yes,Yes,NaN,NaN


That imported ok.  But there are a few extra columns for my needs (I only need it to look up the description for a given term)

In [38]:
# reduce the dataframe to the columns of interest
ks4_meta_df = ks4_meta_df[['Metafile heading', 'Metafile description']]
# relabel them to match my target format
ks4_meta_df.columns = ['label', 'expanded']
# check it looks ok
ks4_meta_df

,label,expanded
0,RECTYPE,Record type (1=mainstream school; 2=special sc...
1,ALPHAIND,Alphabetic sorting index
2,LEA,Local authority code (see separate list of loc...
3,ESTAB,Establishment number
4,URN,School Unique Reference Number
5,SCHNAME,School name
6,SCHNAME_AC,School now known as (used if the school has co...
7,ADDRESS1,School address (1)
8,ADDRESS2,School address (2)
9,ADDRESS3,School address (3)


In [39]:
# set up a reference to the db.collection 
ks4_meta = db.ks4_meta

In [40]:
ks4_meta.insert_many(ks4_meta_df.to_dict('records'))
# snippet reference is from:
# https://stackoverflow.com/questions/33979983/insert-rows-from-pandas-dataframe-into-mongodb-collection-as-individual-document

In [41]:
ks4_meta.find_one({'label': 'NFTYPE'})

{'_id': ObjectId('5b0c7eea0fd01f09d8fcee9f'),
 'expanded': 'School type (see separate list of abbreviations used in the tables)',
 'label': 'NFTYPE'}

I want to add the codes from the abbreviations dictionary to this document since it is one of the backdones to my investigation.

In [42]:
ks4_meta.update_one({'label': 'NFTYPE'}, 
                    {'$set': {'codes': nftypes}})

ks4_meta.find_one({'label': 'NFTYPE'})

{'_id': ObjectId('5b0c7eea0fd01f09d8fcee9f'),
 'codes': {'AC': 'Sponsored academy',
  'AC1619': 'Academy 16-19 sponsor led',
  'ACC': 'Academy converter - mainstream',
  'ACC1619': 'Academy 16-19 converter',
  'ACCS': 'Academy converter - special school',
  'ACS': 'Sponsored special academy',
  'CTC': 'City technology college',
  'CY': 'Community school',
  'CYS': 'Community special school',
  'F': 'Free school - mainstream',
  'F1619': 'Free school - 16-19',
  'FD': 'Foundation school',
  'FDS': 'Foundation special school',
  'FESI': 'Further Education Sector Institution',
  'FS': 'Free school - special',
  'FSS': 'Studio school',
  'FUTC': 'UTC (university technical college)',
  'IND': 'Independent school',
  'INDSPEC': 'Independent special school',
  'MODFC': 'College funded by Ministry of Defence',
  'NMSS': 'Non-maintained special school',
  'VA': 'Voluntary aided school',
  'VC': 'Voluntary controlled school'},
 'expanded': 'School type (see separate list of abbreviations used in

I'll do the same for the `RECTYPE` label by splitting the description.

In [43]:
# select the correct document
r = ks4_meta.find_one({'label': 'RECTYPE'})

# checks that we haven't already updated the document
# then if not splits the description string, adding a code key
# to reference each school type
if 'codes' not in r.keys():
    expanded = r['expanded']
    e = expanded[:11]
    codelist = expanded[13:-1].split('; ')
    keys = [c[:1] for c in codelist]
    values = [c[2:] for c in codelist]
    codes = (dict(list(zip(keys, values))))
    ks4_meta.update_one({'_id': r['_id']},
                        {'$set': {'expanded': e,
                                  'codes': codes}})

# check that it was processed correctly
ks4_meta.find_one({'label': 'RECTYPE'})

{'_id': ObjectId('5b0c7eea0fd01f09d8fcee90'),
 'codes': {'1': 'mainstream school',
  '2': 'special school',
  '4': 'local authority',
  '5': 'National (all schools)',
  '7': 'National (maintained schools)'},
 'expanded': 'Record type',
 'label': 'RECTYPE'}

Great.  That is most of the cleaning I need to do for the ks4_meta file.  If I were to be doing a different investigation I would consider merging in the LEA data here, but we already have it stored from earlier on (importing ks2) as a dataframe which we can reference if needed.

Great.  Now in the tm351 module materials we had some handy collections provided by the module team that enabled us to quickly look up the labels and codes of a given accident.  I'll borrow that idea here for my purposes.  Because, I will need to do the same for the KS2 dataset, I'll wrap them in a function.

In [44]:
# code adapted from the p14 accidents dataset notebooks

def expanded_label(meta):
    # Load the expanded names of keys and human-readable codes into memory
    expanded_name = collections.defaultdict(str)
    for e in meta.find({'expanded': {"$exists": True}}):
        expanded_name[e['label']] = e['expanded']

    label_of = collections.defaultdict(str)
    for l in meta.find({'codes': {"$exists": True}}):
        for c in l['codes']:
            try:
                label_of[l['label'], int(c)] = l['codes'][c]
            except ValueError: 
                label_of[l['label'], c] = l['codes'][c]
    # return both as a tuple
    return (expanded_name, label_of)

In [45]:
# Set up the expanded_name and label_of for ks4_meta
ks4_expanded_name, ks4_label_of = expanded_label(ks4_meta)

In [46]:
# test it works
[(c, ks4_label_of['RECTYPE', c]) for k, c in ks4_label_of if k == 'RECTYPE']

[(1, 'mainstream school'),
 (4, 'local authority'),
 (2, 'special school'),
 (7, 'National (maintained schools)'),
 (5, 'National (all schools)')]

In [47]:
ks4_expanded_name['NFTYPE']

'School type (see separate list of abbreviations used in the tables)'

In [48]:
ks4_label_of['NFTYPE', 'AC']

'Sponsored academy'

Great that all is working.  I'll quickly repeat the same steps for KS2_meta data

In [49]:
# relabel the columns of ks2cols
ks2cols.columns = ['not_needed', 'label', 'expanded']

# create a collection in the database
ks2_meta = db.ks2_meta

In [50]:
# store them into the database
ks2_meta.insert_many(ks2cols[['label', 'expanded']].to_dict('records'))

In [51]:
ks2_meta.find_one()

{'_id': ObjectId('5b0c7eea0fd01f09d8fcf004'),
 'expanded': 'Record type (1=mainstream school; 2=special school; 3=Local Authority; 4=National (all schools); 5=National (maintained schools))',
 'label': 'RECTYPE'}

In [52]:
# repeat the splitting of the `RECTYPE`
# select the correct document
r = ks2_meta.find_one({'label': 'RECTYPE'})

# checks that we haven't already updated the document
# then if not splits the description string, adding a code key
# to reference each school type
if 'codes' not in r.keys():
    expanded = r['expanded']
    e = expanded[:11]
    codelist = expanded[13:-1].split('; ')
    keys = [c[:1] for c in codelist]
    values = [c[2:] for c in codelist]
    codes = (dict(list(zip(keys, values))))
    ks2_meta.update_one({'_id': r['_id']},
                        {'$set': {'expanded': e,
                                  'codes': codes}})

# check that it was processed correctly
ks2_meta.find_one({'label': 'RECTYPE'})

{'_id': ObjectId('5b0c7eea0fd01f09d8fcf004'),
 'codes': {'1': 'mainstream school',
  '2': 'special school',
  '3': 'Local Authority',
  '4': 'National (all schools)',
  '5': 'National (maintained schools)'},
 'expanded': 'Record type',
 'label': 'RECTYPE'}

In [53]:
# And add the nftype to the meta collection
ks2_meta.update_one({'label': 'NFTYPE'}, 
                    {'$set': {'codes': nftypes}})

ks2_meta.find_one({'label': 'NFTYPE'})

{'_id': ObjectId('5b0c7eea0fd01f09d8fcf013'),
 'codes': {'AC': 'Sponsored academy',
  'AC1619': 'Academy 16-19 sponsor led',
  'ACC': 'Academy converter - mainstream',
  'ACC1619': 'Academy 16-19 converter',
  'ACCS': 'Academy converter - special school',
  'ACS': 'Sponsored special academy',
  'CTC': 'City technology college',
  'CY': 'Community school',
  'CYS': 'Community special school',
  'F': 'Free school - mainstream',
  'F1619': 'Free school - 16-19',
  'FD': 'Foundation school',
  'FDS': 'Foundation special school',
  'FESI': 'Further Education Sector Institution',
  'FS': 'Free school - special',
  'FSS': 'Studio school',
  'FUTC': 'UTC (university technical college)',
  'IND': 'Independent school',
  'INDSPEC': 'Independent special school',
  'MODFC': 'College funded by Ministry of Defence',
  'NMSS': 'Non-maintained special school',
  'VA': 'Voluntary aided school',
  'VC': 'Voluntary controlled school'},
 'expanded': 'School type',
 'label': 'NFTYPE'}

In [54]:
# finally, set up the expanded_name and label_of for ks4_meta
ks2_expanded_name, ks2_label_of = expanded_label(ks2_meta)

check they work ok

In [55]:
# test it works
[(c, ks4_label_of['RECTYPE', c]) for k, c in ks4_label_of if k == 'RECTYPE']

[(1, 'mainstream school'),
 (4, 'local authority'),
 (2, 'special school'),
 (7, 'National (maintained schools)'),
 (5, 'National (all schools)')]

In [56]:
ks4_label_of['NFTYPE', 'IND']

'Independent school'

Great that is all the meta data handled, and we can now go about importing the KS4 data into the database and cleaning it.

## Importing the KS4 dataset

In [57]:
!/usr/bin/mongoimport --port 27351 --drop --db schools_db --collection ks4 \
    --type csv --headerline --ignoreBlanks \
    --file data/2015-2016/england_ks4final.csv

2018-05-28T22:12:59.001+0000	connected to: localhost:27351
2018-05-28T22:12:59.001+0000	dropping: schools_db.ks4
2018-05-28T22:13:01.065+0000	imported 5489 documents


In [58]:
ks4 = db.ks4
ks4.find_one()

{'AC5EM13': '0%',
 'AC5EM14_PTQ': '0%',
 'AC5EM15_PTQ_EE': '0%',
 'AC5EM16_PTQ_EE': '0%',
 'ADDRESS1': 'Queen Victoria Street',
 'AGERANGE': '10-18',
 'ALPHAIND': 11828,
 'ATT8SCR': 42.1,
 'ATT8SCREBAC': 22.2,
 'ATT8SCREBAC_FSM6CLA1A': 'NP',
 'ATT8SCREBAC_NFSM6CLA1A': 'NP',
 'ATT8SCRENG': 7.3,
 'ATT8SCRENG_FSM6CLA1A': 'NP',
 'ATT8SCRENG_NFSM6CLA1A': 'NP',
 'ATT8SCRMAT': 0,
 'ATT8SCRMAT_FSM6CLA1A': 'NP',
 'ATT8SCRMAT_NFSM6CLA1A': 'NP',
 'ATT8SCROPEN': 12.6,
 'ATT8SCROPENG': 10.4,
 'ATT8SCROPENG_FSM6CLA1A': 'NP',
 'ATT8SCROPENG_NFSM6CLA1A': 'NP',
 'ATT8SCROPENNG': 2.2,
 'ATT8SCROPENNG_FSM6CLA1A': 'NP',
 'ATT8SCROPENNG_NFSM6CLA1A': 'NP',
 'ATT8SCROPEN_FSM6CLA1A': 'NP',
 'ATT8SCROPEN_NFSM6CLA1A': 'NP',
 'ATT8SCR_15': 'NA',
 'ATT8SCR_AV': 'NP',
 'ATT8SCR_BOYS': 42.1,
 'ATT8SCR_EAL': 'NP',
 'ATT8SCR_FSM6CLA1A': 'NP',
 'ATT8SCR_GIRLS': 'NA',
 'ATT8SCR_HI': 'NP',
 'ATT8SCR_LO': 'NP',
 'ATT8SCR_NFSM6CLA1A': 'NP',
 'ATT8SCR_NMOB': 'NP',
 'BPUP': 139,
 'CONTFLAG': 0,
 'DIFFN_ATT8': 'NP',
 'DIFFN_

# Cleanup/remove the database
<a name="cleanup"></a>

Uncomment the lines below to remove the MongoDB created in the investigation.

In [59]:
# uncomment to remove the database if needed
# client.drop_database('schools_db')
# client.database_names()

['accidents', 'admin', 'local']